In [1]:
import math, time

In [4]:
def pos_rc(i: int, n: int):
    return divmod(i, n)

def pos_ix(r: int, c: int, n: int):
    return r * n + c

print("pos_rc(5,3) =", pos_rc(5,4))
print("pos_ix(2,1,3) =", pos_ix(2,1,3))

pos_rc(5,3) = (1, 1)
pos_ix(2,1,3) = 7


In [5]:
def parse_puzzle(text: str):
    lines = [ln.strip() for ln in text.strip().splitlines() if ln.strip()]
    N = int(lines[0]) #number of numbered tiles
    I = int(lines[1]) #goal index where 0 must be placed, -1 is default for bottom left
    total = N + 1 #number of total tiles (numbered + empty one)
    n = int(math.isqrt(total))
    assert n*n == total
    board = []
    for r in range(n):
        row = list(map(int, lines[2+r].split()))
        assert len(row) == n
        board += row
    if I == -1:
        I = n*n - 1
    return N, I, tuple(board), n

In [6]:
def build_goal(N: int, n: int, I: int):
    g = [None]*(n*n)
    g[I] = 0
    t = 1
    for k in range(n*n):
        if g[k] is None:
            g[k] = t
            t += 1
    return tuple(g)

In [9]:
demo_text = """
8
-1
1 2 3 
4 5 6 
0 7 8 
"""
N, I, start, n = parse_puzzle(demo_text)
goal = build_goal(N, n, I)
print("Parsed:", (N, I, n), "start:", start)
print("Goal  :", goal)

Parsed: (8, 8, 3) start: (1, 2, 3, 4, 5, 6, 0, 7, 8)
Goal  : (1, 2, 3, 4, 5, 6, 7, 8, 0)


In [10]:
def dest_rc_for_tile(t: int, n: int, I: int):
    flat = (t - 1) if (t <= I) else t
    return pos_rc(flat, n)

def h_manhattan(state, n: int, I: int):
    dist = 0
    for i, t in enumerate(state):
        if t == 0: 
            continue
        r, c = pos_rc(i, n)
        gr, gc = dest_rc_for_tile(t, n, I)
        dist += abs(r - gr) + abs(c - gc)
    return dist

print("h_manhattan(start) =", h_manhattan(start, n, I))
print("Goal dest for tile 8:", dest_rc_for_tile(5, n, I))

h_manhattan(start) = 2
Goal dest for tile 8: (1, 1)


In [12]:
def inversions_parity(seq):
    p = 0
    for i in range(len(seq)):
        for j in range(i+1, len(seq)):
            p ^= (seq[i] > seq[j])
    return p

def row_from_bottom(idx: int, n: int):
    r, _ = pos_rc(idx, n)
    return n - r

def is_solvable(state, n: int, I: int):
    nums = [x for x in state if x != 0]
    inv_s = inversions_parity(nums)
    inv_g = 0
    z_s = state.index(0)
    z_g = I
    if n % 2 == 1:
        return inv_s == inv_g
    else:
        return (inv_s + row_from_bottom(z_s, n)) % 2 == (inv_g + row_from_bottom(z_g, n)) % 2

unsolvable_3x3 = (1,3,2,4,6,5,7,8,0)
print("Unsolvable example (3x3)?", is_solvable(unsolvable_3x3, 3, 8))
print("Our demo solvable?       ", is_solvable(start, n, I))

Unsolvable example (3x3)? True
Our demo solvable?        True


In [13]:
MOVES = {
    'U': (-1, 0, 'down'),  # up => means that the empty tile has gone down
    'D': ( 1, 0, 'up'),
    'L': ( 0,-1, 'right'),
    'R': ( 0, 1, 'left'),
}

# avoids immediate backtracking by undoing last move
OPP = {'U':'D',
       'D':'U',
       'L':'R',
       'R':'L'} 

In [14]:
def ida_solve(start, goal, n: int, I: int, time_limit_sec=None):
    # Returns list of tile directions or None
    
    threshold = h_manhattan(start, n, I)
    z0 = start.index(0)
    t0 = time.time()

    def dfs(state, g, limit, z, prev_blank_move, path, visited):
        f = g + h_manhattan(state, n, I)
        if f > limit:
            return f
        if state == goal:
            return "OK"

        best_cut = float('inf')
        zr, zc = pos_rc(z, n)

        for code, (dr, dc, out_dir) in MOVES.items():
            if prev_blank_move and code == OPP[prev_blank_move]:
                continue  # does not immediately undo the last move
            nr, nc = zr + dr, zc + dc
            if not (0 <= nr < n and 0 <= nc < n): 
                continue
            k = pos_ix(nr, nc, n)
            arr = list(state)
            arr[z], arr[k] = arr[k], arr[z]
            ns = tuple(arr)
            if ns in visited:
                continue
            visited.add(ns); path.append(out_dir)
            res = dfs(ns, g+1, limit, k, code, path, visited)
            if res == "OK":
                return "OK"
            if res < best_cut:
                best_cut = res
            path.pop(); visited.remove(ns)
        return best_cut

    while True:
        path, visited = [], {start}
        res = dfs(start, 0, threshold, z0, None, path, visited)
        if res == "OK":
            return path
        if res == float('inf'):
            return None
        threshold = res
        if time_limit_sec and (time.time() - t0) > time_limit_sec:
            return None

In [15]:
near_goal = (1,2,0,3,4,5,6,7,8)
goal_3x3  = build_goal(8, 3, 8)
print("IDA* path from near_goal:", ida_solve(near_goal, goal_3x3, 3, 8))

IDA* path from near_goal: ['up', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'up', 'left', 'left']


In [32]:
demo_text = """
15
-1
1 2 3 4
5 6 7 8
9 12 11 10
14 13 15 0
"""

In [33]:
DIR_TO_BLANK = {'left':'R', 'right':'L', 'up':'D', 'down':'U'}

In [34]:
def apply_step(state, n: int, tile_direction: str):
    code = DIR_TO_BLANK[tile_direction]
    z = state.index(0)
    zr, zc = pos_rc(z, n)
    dr, dc, _ = MOVES[code]
    nr, nc = zr + dr, zc + dc
    k = pos_ix(nr, nc, n)
    arr = list(state)
    arr[z], arr[k] = arr[k], arr[z]
    return tuple(arr)

def print_board(state, n: int):
    w = len(str(n*n-1))
    for r in range(n):
        row = state[r*n:(r+1)*n]
        print(" ".join(f"{x:>{w}}" for x in row))
    print()

def replay(text: str):
    N, I, start, n = parse_puzzle(text)
    goal = build_goal(N, n, I)

    if not is_solvable(start, n, I):
        print("Unsolvable.")
        return

    path = ida_solve(start, goal, n, I)
    cur = start
    print("Start:"); print_board(cur, n)
    for step in path:
        cur = apply_step(cur, n, step)
        print(step); print_board(cur, n)

In [35]:
replay(demo_text)

Start:
 1  2  3  4
 5  6  7  8
 9 12 11 10
14 13 15  0

down
 1  2  3  4
 5  6  7  8
 9 12 11  0
14 13 15 10

right
 1  2  3  4
 5  6  7  8
 9 12  0 11
14 13 15 10

up
 1  2  3  4
 5  6  7  8
 9 12 15 11
14 13  0 10

right
 1  2  3  4
 5  6  7  8
 9 12 15 11
14  0 13 10

right
 1  2  3  4
 5  6  7  8
 9 12 15 11
 0 14 13 10

down
 1  2  3  4
 5  6  7  8
 0 12 15 11
 9 14 13 10

left
 1  2  3  4
 5  6  7  8
12  0 15 11
 9 14 13 10

up
 1  2  3  4
 5  6  7  8
12 14 15 11
 9  0 13 10

left
 1  2  3  4
 5  6  7  8
12 14 15 11
 9 13  0 10

left
 1  2  3  4
 5  6  7  8
12 14 15 11
 9 13 10  0

down
 1  2  3  4
 5  6  7  8
12 14 15  0
 9 13 10 11

right
 1  2  3  4
 5  6  7  8
12 14  0 15
 9 13 10 11

right
 1  2  3  4
 5  6  7  8
12  0 14 15
 9 13 10 11

right
 1  2  3  4
 5  6  7  8
 0 12 14 15
 9 13 10 11

up
 1  2  3  4
 5  6  7  8
 9 12 14 15
 0 13 10 11

left
 1  2  3  4
 5  6  7  8
 9 12 14 15
13  0 10 11

left
 1  2  3  4
 5  6  7  8
 9 12 14 15
13 10  0 11

down
 1  2  3  4
 5  6  7 